<a href="https://colab.research.google.com/github/AFBA1993/Projects/blob/main/TemporalSeries/PAGS/PAGS_LSTM_Func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io
from keras.metrics import mean_squared_error
from keras.models import load_model

In [37]:
def LSTM_PAGS(LOOK_BACK, DROPUT, NEURONS, EPOCHS, BATCH_SIZE ):

  # Loading PAGS raw database:
  url = ('https://raw.githubusercontent.com/AFBA1993/Projects/main/'+
       'TemporalSeries/PAGS/PAGSattempt.csv')
  url_data = requests.get(url).content
  raw_data = pd.read_csv(io.StringIO(url_data.decode('utf-8')))
  raw_data = raw_data[["Date","Close"]]
  raw_data.index = pd.to_datetime(raw_data["Date"])
  raw_data.drop("Date", axis = 1, inplace =  True)

  #Spliting data into train and test sections:
  SPLIT_PERCENT = 0.80
  split = int(SPLIT_PERCENT*len(raw_data))
  train = raw_data[:split]
  test = raw_data[split:]

  #Shaping Training Data:
  input_train = train.values
  input_train = input_train.reshape((-1, 1))
  scaler = MinMaxScaler(feature_range = (0,1))
  input_train = scaler.fit_transform(input_train)

   
  X_train = []
  y_train = []

  for i in range (LOOK_BACK, len(input_train)):
    X_train.append(input_train[i-LOOK_BACK:i,0])
    y_train.append(input_train[i,0])

  X_train, y_train = np.array(X_train), np.array(y_train)
  X_train = np.reshape(X_train, (X_train.shape[0], LOOK_BACK, 1))

  #Configuring LSTM:

  model = Sequential()
  model.add(LSTM(units = NEURONS, return_sequences = True,
                    input_shape = (X_train.shape[1], 1)))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))

  model.add(LSTM(units = NEURONS, return_sequences = True))
  model.add(Dropout(DROPUT))  

  model.add(LSTM(units = NEURONS))
  model.add(Dropout(DROPUT))


  model.add(Dense(units = 1, activation = 'linear'))
  model.compile(optimizer = 'rmsprop', loss = 'mse',
                    metrics = ['mse'])
  
  model.fit(X_train, y_train, epochs = EPOCHS, batch_size = BATCH_SIZE, 
                                                            workers=-1)

  #Testing LSTM in the train section:
  LSTM_train = model.predict(X_train)
  LSTM_train = scaler.inverse_transform(LSTM_train)
  LSTM_train = pd.DataFrame(LSTM_train, columns = ['Close'])
  LSTM_train["Date"] = train[LOOK_BACK:].index
  LSTM_train.index = pd.to_datetime(LSTM_train["Date"])
  LSTM_train.drop("Date", axis = 1, inplace =  True)

  train_mse = np.reshape(train.values, (train.values.shape[0]))
  LSTM_train_mse = np.reshape(LSTM_train.values, (LSTM_train.values.shape[0]))
  MSE_train = mean_squared_error(train_mse[LOOK_BACK:], LSTM_train_mse)
  
  #Testing LSTM in the train section:
  complete_data = raw_data.values
  input_test = complete_data[len(complete_data) - len(test) - LOOK_BACK:]
  input_test = input_test.reshape((-1, 1))
  input_test = scaler.transform(input_test)

  X_test = []
  for i in range(LOOK_BACK, len(input_test)):
    X_test.append(input_test[i-LOOK_BACK:i,0])
  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

  LSTM_test = model.predict(X_test)
  LSTM_test = scaler.inverse_transform(LSTM_test)

  LSTM_test = pd.DataFrame(LSTM_test, columns = ['Close'])
  LSTM_test["Date"] = test.index
  LSTM_test.index = pd.to_datetime(LSTM_test["Date"])
  LSTM_test.drop("Date", axis = 1, inplace =  True) 

  test_mse = np.reshape(test.values, (test.values.shape[0]))
  LSTM_test_mse = np.reshape(LSTM_test.values, (LSTM_test.values.shape[0]))
  MSE_test = mean_squared_error(test_mse, LSTM_test_mse)

  return MSE_train, MSE_test, model

In [38]:
def LSTM_PAGS_VAL(LOOK_BACK, model):

  # Loading PAGS raw database:
  url = ('https://raw.githubusercontent.com/AFBA1993/Projects/main/'+
       'TemporalSeries/PAGS/PAGSattempt.csv')
  url_data = requests.get(url).content
  raw_data = pd.read_csv(io.StringIO(url_data.decode('utf-8')))
  raw_data = raw_data[["Date","Close"]]
  raw_data.index = pd.to_datetime(raw_data["Date"])
  raw_data.drop("Date", axis = 1, inplace =  True)

  #Spliting data into train and test sections:
  SPLIT_PERCENT = 0.80
  split = int(SPLIT_PERCENT*len(raw_data))
  train = raw_data[:split]
  test = raw_data[split:]

  #Shaping Training Data:
  input_train = train.values
  input_train = input_train.reshape((-1, 1))
  scaler = MinMaxScaler(feature_range = (0,1))
  input_train = scaler.fit_transform(input_train)

   
  X_train = []
  y_train = []

  for i in range (LOOK_BACK, len(input_train)):
    X_train.append(input_train[i-LOOK_BACK:i,0])
    y_train.append(input_train[i,0])

  X_train, y_train = np.array(X_train), np.array(y_train)
  X_train = np.reshape(X_train, (X_train.shape[0], LOOK_BACK, 1))

  #Testing LSTM in the train section:
  LSTM_train = model.predict(X_train)
  LSTM_train = scaler.inverse_transform(LSTM_train)
  LSTM_train = pd.DataFrame(LSTM_train, columns = ['Close'])
  LSTM_train["Date"] = train[LOOK_BACK:].index
  LSTM_train.index = pd.to_datetime(LSTM_train["Date"])
  LSTM_train.drop("Date", axis = 1, inplace =  True)

  train_mse = np.reshape(train.values, (train.values.shape[0]))
  LSTM_train_mse = np.reshape(LSTM_train.values, (LSTM_train.values.shape[0]))
  MSE_train = mean_squared_error(train_mse[LOOK_BACK:], LSTM_train_mse)
  
  #Testing LSTM in the train section:
  complete_data = raw_data.values
  input_test = complete_data[len(complete_data) - len(test) - LOOK_BACK:]
  input_test = input_test.reshape((-1, 1))
  input_test = scaler.transform(input_test)

  X_test = []
  for i in range(LOOK_BACK, len(input_test)):
    X_test.append(input_test[i-LOOK_BACK:i,0])
  X_test = np.array(X_test)
  X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1],1))

  LSTM_test = model.predict(X_test)
  LSTM_test = scaler.inverse_transform(LSTM_test)

  LSTM_test = pd.DataFrame(LSTM_test, columns = ['Close'])
  LSTM_test["Date"] = test.index
  LSTM_test.index = pd.to_datetime(LSTM_test["Date"])
  LSTM_test.drop("Date", axis = 1, inplace =  True) 

  test_mse = np.reshape(test.values, (test.values.shape[0]))
  LSTM_test_mse = np.reshape(LSTM_test.values, (LSTM_test.values.shape[0]))
  MSE_test = mean_squared_error(test_mse, LSTM_test_mse)

  return MSE_train, MSE_test, model

In [39]:
LOOK_BACK = 150
DROPUT = 0.1
NEURONS = 35
EPOCHS = 500
BATCH_SIZE = 128

model = LSTM_PAGS(LOOK_BACK, DROPUT, NEURONS, EPOCHS, BATCH_SIZE )



Epoch 1/500
6/6 [==============================] - 21s 808ms/step - loss: 0.1403 - mse: 0.1403
Epoch 2/500
6/6 [==============================] - 5s 813ms/step - loss: 0.0298 - mse: 0.0298
Epoch 3/500
6/6 [==============================] - 5s 809ms/step - loss: 0.0429 - mse: 0.0429
Epoch 4/500
6/6 [==============================] - 5s 796ms/step - loss: 0.0270 - mse: 0.0270
Epoch 5/500
6/6 [==============================] - 5s 819ms/step - loss: 0.0394 - mse: 0.0394
Epoch 6/500
6/6 [==============================] - 5s 854ms/step - loss: 0.0372 - mse: 0.0372
Epoch 7/500
6/6 [==============================] - 6s 1s/step - loss: 0.0214 - mse: 0.0214
Epoch 8/500
6/6 [==============================] - 5s 824ms/step - loss: 0.0339 - mse: 0.0339
Epoch 9/500
6/6 [==============================] - 5s 820ms/step - loss: 0.0245 - mse: 0.0245
Epoch 10/500
6/6 [==============================] - 5s 808ms/step - loss: 0.0222 - mse: 0.0222
Epoch 11/500
6/6 [==============================] - 5s 815ms/

In [41]:
model

(<tf.Tensor: shape=(), dtype=float32, numpy=2.0361495>,
 <tf.Tensor: shape=(), dtype=float32, numpy=73.4287>,
 <keras.engine.sequential.Sequential at 0x7f29ba1370d0>)

In [43]:
model[2].save("model_4.h5")

In [44]:
MODEL = load_model('model_4.h5')

In [45]:
MODEL.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_19 (LSTM)              (None, 150, 35)           5180      
                                                                 
 dropout_19 (Dropout)        (None, 150, 35)           0         
                                                                 
 lstm_20 (LSTM)              (None, 150, 35)           9940      
                                                                 
 dropout_20 (Dropout)        (None, 150, 35)           0         
                                                                 
 lstm_21 (LSTM)              (None, 150, 35)           9940      
                                                                 
 dropout_21 (Dropout)        (None, 150, 35)           0         
                                                                 
 lstm_22 (LSTM)              (None, 150, 35)          

In [46]:
LSTM_PAGS_VAL(150, MODEL)

(<tf.Tensor: shape=(), dtype=float32, numpy=2.0361495>,
 <tf.Tensor: shape=(), dtype=float32, numpy=73.4287>,
 <keras.engine.sequential.Sequential at 0x7f29b9081d50>)